In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec 28 20:57:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:0A:00.0  On |                  N/A |
|  0%   31C    P8    36W / 390W |   1401MiB / 24576MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#%%capture
#!pip install datasets==1.13.3
#!pip install transformers==4.11.3
#!pip install huggingface_hub==0.1
#!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
#!pip install jiwer

In [3]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_10_0", "uk", split="train", use_auth_token=True)
common_voice_test = load_dataset("mozilla-foundation/common_voice_10_0", "uk", split="test", use_auth_token=True)

Reusing dataset common_voice_10_0 (/home/robinhad/.cache/huggingface/datasets/mozilla-foundation___common_voice_10_0/uk/10.0.0/27df768ab1b5cac48a7616f145b79b62599167b0ffa2e054bf4c3e74e9619e5e)
Reusing dataset common_voice_10_0 (/home/robinhad/.cache/huggingface/datasets/mozilla-foundation___common_voice_10_0/uk/10.0.0/27df768ab1b5cac48a7616f145b79b62599167b0ffa2e054bf4c3e74e9619e5e)


In [4]:
common_voice_train

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 11463
})

In [5]:
common_voice_train.cleanup_cache_files()
common_voice_test.cleanup_cache_files()

0

In [6]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,У червоних — невдачі на фронті.
1,Він нагадував недавні ночі в кам'янському парку.
2,Творення займенників
3,"Коли бідний жениться, ніч мала."
4,Чорнота і Бугай злізли з дерев і пішли назирцем.
5,крик.
6,"Крім того, мало не завжди погода примушує його десь заночувати."
7,Така вже мода тепер.
8,Летить що має сили до вікна і — грим грудьми до шибки.
9,"Ворожа лава проминула вже балку, а Василенко не стріляв."


In [9]:
filter_func = lambda x: not ("joki" in x or "ы" in x)
common_voice_train = common_voice_train.filter(filter_func, input_columns=["sentence"])
common_voice_test = common_voice_test.filter(filter_func, input_columns=["sentence"])

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [10]:
def cleaner(batch):
    replace_as_space = "!:;,—…–“”?\"«»"
    special_words = {
        "ХIХ": "дев'ятнадцятого",
        "Linux": "Лінукс",
        "Maace": "Маасе",
        "м 'ясо": "м'ясо",
        "'іде": "іде",
        "Д'Аламбер": "даламбер",
        " - ": " ",
        "--": " ",
        "....": " ",
        "...": " ",
        "..": " ",
        " '": " ",
        "О'": "о",
        "-": " " #further check needed
    }
    # check abbreviations later
    abbreviations = {
        'ЧК': "чека",
        'ҐПУ': "ґепеу",
        'ЄС.': "єес",
        'УНР': "уенер",
        'ДТП.': "детепе",
        'РНБО': "еренбео",
        'СРСР': "есересер",
        'ДБР': "дебеер",
        'КП': "капе",
        'ОС': "оес",
    } 
    chars_dict = {
        "C": "С",
        "I": "І",
        "P": "Р",
        "a": "а",
        "e": "е",
        "x": "х",
        "y": "у",
        "p": "р",
        "o": "о",
        "i": "і",
        "\u0301": "",
        "`": "'",
        "՚": "'",
        ".": " ",
        "’": "'"
        
    }
    for word in special_words.keys():
        batch["sentence"] = batch["sentence"].replace(word, special_words[word])
    for word in abbreviations.keys():
        batch["sentence"] = batch["sentence"].replace(word, abbreviations[word])
    for char in chars_dict.keys():
        batch["sentence"] = batch["sentence"].replace(char, chars_dict[char])
    for char in replace_as_space:
        batch["sentence"] = batch["sentence"].replace(char, " ")
    batch["sentence"] = " ".join(filter(lambda x: x != "", batch["sentence"].strip().lower().split(" ")))
    return batch

sentence = {"sentence": "Привіт, - як у тебе справи загалом, м'якотілий друже?"}
cleaner(sentence)

{'sentence': "привіт як у тебе справи загалом м'якотілий друже"}

In [11]:
show_random_elements(common_voice_train.map(cleaner).remove_columns(["path", "audio"]), num_examples=10)

  0%|          | 0/11463 [00:00<?, ?ex/s]

,sentence
0,отаманенко почав пояснювати з наукової точки але дід перебив його
1,енею глуздівно сказав
2,ні розвідки вперед ні стежі до лісу
3,ну ну та я нічого не кажу
4,якось прийшов зв'язковий із мельників
5,я хоч не з мельників так мені оповідав батько був козаком у холодному яру
6,ну бо
7,макітру одділив од плеч
8,видно стріляла розвідка ударників що йшла із собакою попереду групи
9,левко слабий лежить просить щоб зайшов


In [12]:
common_voice_train = common_voice_train.map(cleaner)
common_voice_test = common_voice_test.map(cleaner)

Loading cached processed dataset at /home/robinhad/.cache/huggingface/datasets/mozilla-foundation___common_voice_10_0/uk/10.0.0/27df768ab1b5cac48a7616f145b79b62599167b0ffa2e054bf4c3e74e9619e5e/cache-96af4ec6cf30f0d6.arrow


  0%|          | 0/6783 [00:00<?, ?ex/s]

In [13]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,чому алгоритм зупиниться
1,конем
2,наступного дня нас прийняли на службу до міліції
3,я знав що це неправда
4,і взявши з запічка кресало
5,скоріше б на гору бо тачанки ар'єргарду вже відкрили вогонь
6,удень відтягалася вглиб села залишаючи наглядати за виходом із міста неозброєних
7,чи співвідноситься це твердження з поняттям карми
8,селяни мусили шукати бодай якоїсь їжі
9,більшість убитих і полонених були одягнені в селянські кожухи


In [14]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [15]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [17]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 "'": 1,
 'а': 2,
 'б': 3,
 'в': 4,
 'г': 5,
 'д': 6,
 'е': 7,
 'ж': 8,
 'з': 9,
 'и': 10,
 'й': 11,
 'к': 12,
 'л': 13,
 'м': 14,
 'н': 15,
 'о': 16,
 'п': 17,
 'р': 18,
 'с': 19,
 'т': 20,
 'у': 21,
 'ф': 22,
 'х': 23,
 'ц': 24,
 'ч': 25,
 'ш': 26,
 'щ': 27,
 'ь': 28,
 'ю': 29,
 'я': 30,
 'є': 31,
 'і': 32,
 'ї': 33,
 'ґ': 34}

In [18]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [19]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

37

In [20]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [21]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
repo_name = "wav2vec2-xls-r-base-uk"

In [23]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [24]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [25]:
# save tokenizer to folder
processor.save_pretrained(repo_name)

In [26]:
common_voice_train[0]["path"]

'/home/robinhad/.cache/huggingface/datasets/downloads/extracted/ee7155196e5d51620d53e48cf58eb693b7839b8ff183604c8bb948d3e0aad92d/cv-corpus-10.0-2022-07-04/uk/clips/common_voice_uk_20907128.mp3'

In [27]:
common_voice_train[0]["audio"]

{'path': '/home/robinhad/.cache/huggingface/datasets/downloads/extracted/ee7155196e5d51620d53e48cf58eb693b7839b8ff183604c8bb948d3e0aad92d/cv-corpus-10.0-2022-07-04/uk/clips/common_voice_uk_20907128.mp3',
 'array': array([ 0.0000000e+00, -3.5002383e-14,  9.4785833e-15, ...,
        -5.0386465e-08, -4.4114326e-08, -1.9402206e-08], dtype=float32),
 'sampling_rate': 48000}

In [28]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [29]:
common_voice_train[0]["audio"]

{'path': '/home/robinhad/.cache/huggingface/datasets/downloads/extracted/ee7155196e5d51620d53e48cf58eb693b7839b8ff183604c8bb948d3e0aad92d/cv-corpus-10.0-2022-07-04/uk/clips/common_voice_uk_20907128.mp3',
 'array': array([ 1.00456624e-13, -1.54340042e-13,  7.00158518e-13, ...,
        -1.50335762e-08, -1.92623926e-08, -2.21930367e-08], dtype=float32),
 'sampling_rate': 16000}

In [30]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

здогадалась дівочити по семій дитині


In [31]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: там і стали на ночівлю
Input array shape: (36288,)
Sampling rate: 16000


In [32]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [33]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

  0%|          | 0/11463 [00:00<?, ?ex/s]

  0%|          | 0/6783 [00:00<?, ?ex/s]

In [34]:
#max_input_length_in_sec = 5.0
#common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

In [35]:
!mkdir cached_dataset

mkdir: cannot create directory ‘cached_dataset’: File exists


In [36]:
common_voice_train.save_to_disk("cached_dataset/cv_train")

In [37]:
common_voice_test.save_to_disk("cached_dataset/cv_test")